In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
from Bio import SeqIO


2025-03-03 23:25:30.623195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 23:25:30.628952: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 23:25:30.642256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741024530.663743   20314 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741024530.670031   20314 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 23:25:30.694945: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
input_file = "../data/ACE2/reduced_ACE2.fasta"
output_file = "../data/ACE2/clean_reduced_ACE2.fasta"

with open(output_file, "w") as out_f:
    for record in SeqIO.parse(input_file, "fasta"):
        record.seq = record.seq.replace("-","")  # Remove gaps
        SeqIO.write(record, out_f, "fasta")

print("Cleaned FASTA saved as", output_file)

Cleaned FASTA saved as ../data/ACE2/clean_reduced_ACE2.fasta


In [4]:
import numpy as np
from Bio import SeqIO
import os
import pickle

# Input and output paths
INPUT_FASTA = "../data/ACE2/clean_reduced_ACE2.fasta"
OUTPUT_FEATURES = "onehot_features.pkl"

# Define amino acid vocabulary
AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY"  # 20 standard amino acids
AA_TO_INDEX = {aa: i for i, aa in enumerate(AMINO_ACIDS)}

# Function to one-hot encode a sequence
def one_hot_encode(seq, max_length):
    encoded = np.zeros((max_length, len(AMINO_ACIDS)), dtype=np.float32)
    for i, aa in enumerate(seq):
        if aa in AA_TO_INDEX:
            encoded[i, AA_TO_INDEX[aa]] = 1
    return encoded

# Main feature extraction function
def extract_features(input_fasta, output_file):
    sequences = [str(record.seq) for record in SeqIO.parse(input_fasta, "fasta")]
    max_length = max(len(seq) for seq in sequences)  # Pad to longest sequence

    features = np.array([one_hot_encode(seq, max_length) for seq in sequences])

    # Save features
    with open(output_file, "wb") as f:
        pickle.dump(features, f)

    print(f"✅ Extracted features saved to {output_file} | Shape: {features.shape}")

if __name__ == "__main__":
    if not os.path.exists(INPUT_FASTA):
        print(f"❌ ERROR: {INPUT_FASTA} not found!")
    else:
        extract_features(INPUT_FASTA, OUTPUT_FEATURES)


✅ Extracted features saved to onehot_features.pkl | Shape: (528, 7093, 20)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
from Bio import SeqIO

# Load extracted features
with open("onehot_features.pkl", "rb") as f:
    X = pickle.load(f)  # Shape: (528, 7093, 20)

# Dummy labels (Replace with real labels if available)
y = np.random.randint(0, 2, size=(X.shape[0],))  # Binary labels (0 or 1)

# Define CNN Model
model = keras.Sequential([
    layers.Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(X.shape[1], X.shape[2])),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=128, kernel_size=3, activation="relu"),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # Binary classification output
])

# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train model
model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

# Save model
model.save("ACE2_mutation_predictor.h5")
print("✅ Model training complete. Saved as ACE2_mutation_predictor.h5")
